In [16]:
import os
import pickle
import glob
from datetime import datetime, timedelta
from konlpy.tag import Okt
import re
from tqdm import tqdm

def load_stopwords(file_path):
    """
    주어진 경로에서 불용어를 불러오는 함수
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords

def preprocess_text(text, stopwords=None):
    """
    Okt를 사용한 텍스트 전처리 함수
    - 불용어 제거
    - 명사 추출
    """
    okt = Okt()
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    
    nouns = okt.nouns(text)
    
    if stopwords:
        nouns = [word for word in nouns if word not in stopwords]
    
    nouns = [word for word in nouns if len(word) > 1]
    
    processed_text = ' '.join(nouns)
    
    return processed_text

def load_and_merge_section_data(section_number, hours_back=9):
    """
    섹션 번호에 해당하는 모든 pkl 파일을 불러와서 기사 본문을 하나의 변수로 병합하는 함수
    현재 시간에서 -hours_back 시간 전까지의 파일만 병합
    """
    current_time = datetime.now()
    start_time = current_time - timedelta(hours=hours_back)
    
    valid_hours = []
    for hour in range(hours_back + 1):  # 0부터 hours_back까지의 시간을 계산
        valid_hour = (start_time + timedelta(hours=hour)).strftime('%H')
        valid_hours.append(valid_hour)
    
    file_patterns = [f"../data/{section_number}/*_{hour}.pkl" for hour in valid_hours]
    
    merged_content = []
    for file_pattern in tqdm(file_patterns, desc=f"섹션 번호: {section_number} 파일"):
        file_list = glob.glob(file_pattern)
        
        for file_path in file_list:
            with open(file_path, 'rb') as file:
                data = pickle.load(file)
                for article in data:
                    merged_content.append(article['content'])
    
    merged_document = ' '.join(merged_content)
    
    return merged_document

def process_sections(section_numbers, stopwords_file_path, hours_back=9):
    """
    주어진 섹션 번호 리스트에 대해 각 섹션별로 데이터를 병합하고 전처리하는 함수
    """
    # 불용어 로드
    print("Loading stopwords...")
    stopwords = load_stopwords(stopwords_file_path)
    
    processed_documents = {}
    
    for section_number in tqdm(section_numbers, desc="Processing sections"):
        # 데이터 병합
        merged_content = load_and_merge_section_data(section_number, hours_back)
        
        # 텍스트 전처리
        print(f"{section_number}번 텍스트 전처리 중...")
        processed_text = preprocess_text(merged_content, stopwords)
        
        # 섹션 번호를 키로 하여 전처리된 문서를 저장
        processed_documents[section_number] = processed_text
    
    return processed_documents

# 사용 예시
section_numbers = [100, 101, 102, 103, 104, 105]
stopwords_file_path = '../data/korean_stopwords.txt'
processed_docs = process_sections(section_numbers, stopwords_file_path)

# 전처리된 각 섹션의 첫 500자를 출력 (확인용)
for section, doc in processed_docs.items():
    print(f"Section {section}: {doc[:500]}")


Loading stopwords...


섹션 번호: 100 파일: 100%|██████████| 10/10 [00:00<00:00, 3641.84it/s]

100번 텍스트 전처리 중입니다



섹션 번호: 101 파일: 100%|██████████| 10/10 [00:00<00:00, 2162.68it/s]


101번 텍스트 전처리 중입니다


...

104번 텍스트 전처리 중입니다


섹션 번호: 105 파일: 100%|██████████| 10/10 [00:00<00:00, 5500.01it/s]


105번 텍스트 전처리 중입니다


Processing sections: 100%|██████████| 6/6 [01:04<00:00, 10.67s/it]

Section 100: 불사 현장 비례 요청 거부 이준석 개혁 신당 의원 김영선 국민 의원 뉴스 서울 뉴스 박태훈 선임 기자 이준석 개혁 신당 의원 김건희 여사 공천 개입 의혹 대해 입장 정리 발표 의원 취재 요청 통해 주요 쟁점 대한 생각 경남 하동 불사 김영선 국민 의원 동한 대해 의원 오전 김영선 의원 관계자 의원 직접 보라 종용 김영선 의원 서울 오기 새벽 불사 도착 대화 새벽 불사 서울 불사 당시 상황 대해 텔레 그램 문자 내용 완결 의원 비례대표 달라 의원 측요구 현장 거부 오전 해당 관계자 생각 재차 오후 금태섭 의원 종로 선거 사무소 개소식 개혁 신당 관계자 해당 내용 공유 결과 부정 김영선 의원 김종인 관리 위원장 가족 비례대표 공천 이야기 개혁 신당 비례대표 공천 약속 방증 주장 뉴스 토마토 텔레 그램 캡처 가지 내용 불확실 성도 계속 이야기 보도 강행 의원 대표 시절 창원 의창 보궐선거 김영선 의원 공천 대해 당시 윤상현 관리 위원장 보궐선거 공천 일임 관여 의원 공천 이유 관위 경상남
Section 101: 지난 오후 서울 여의도 한국 거래소 전광판 하이닉스 종가 뉴스 김영 미래에셋 증권 연구원 하이닉스 인공 지능 핵심 반도체 대역폭 메모리 수요 성장 지속 판단 연구원 하이닉스 대한 투자 의견 매수 목표 주가 만원 유지 겨울 가장 겨울 표현 미국 투자 은행 모건스탠리 보고서 염두 모건스탠리 추석 연휴 지난 겨울 제목 보고서 하이닉스 대한 투자 의견 비중 축소 단계 목표 가도 만원 만원 하향 조정 메모리 반도체 수요 정점 역시 공급 과잉 가능성 이유 모건스탠리 보고서 여파 전날 하이닉스 주가 만원 낙폭 기록 연구원 하이닉스 올해 분기 제품 하량 시장 대치 밑돌 예상 연구원 하이닉스 최근 주가 하락 업황 의미 변화 관성 사고 급진 수익 실현 결과 연구원 하이닉스 수익 예상 하이닉스 노드 성능 검증 노드 개발 연내 마무리 전망 경쟁 안정 공정 미세 화로 성능 제조 원가 측면 우위 지속 가능성 연구원 세대 경우 개발 막바지 단계 분기 출하 엔비디

In [17]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

def prepare_data_for_lda(processed_docs):
    """
    LDA를 위한 데이터를 준비하는 함수
    - processed_docs: 섹션별 전처리된 텍스트 데이터 (딕셔너리 형태)
    """
    # 각 섹션의 전처리된 문서를 리스트로 변환
    texts = []
    for doc in tqdm(processed_docs.values(), desc="Processing Documents"):
        texts.append(doc.split())
    
    # 딕셔너리 생성
    dictionary = corpora.Dictionary(texts)
    
    # 코퍼스 생성 (단어의 빈도수로 변환된 텍스트 데이터)
    corpus = [dictionary.doc2bow(text) for text in tqdm(texts, desc="Creating Corpus")]
    
    return dictionary, corpus

def train_lda_model(dictionary, corpus, num_topics=5, passes=15):
    """
    LDA 모델을 학습하는 함수
    - dictionary: Gensim의 Dictionary 객체
    - corpus: Gensim의 코퍼스 객체
    - num_topics: 생성할 토픽의 수
    - passes: 학습 반복 횟수
    """
    # LDA 모델 학습
    print("Training LDA Model...")
    lda_model = LdaModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics,
                         random_state=100,
                         update_every=1,
                         passes=passes,
                         alpha='auto',
                         per_word_topics=True)
    
    return lda_model

def print_lda_topics(lda_model, num_words=10):
    """
    LDA 모델로부터 생성된 토픽들을 출력하는 함수
    - lda_model: 학습된 LDA 모델
    - num_words: 각 토픽에 대해 출력할 단어 수
    """
    print("토픽 추출 중...")
    topics = lda_model.show_topics(num_topics=-1, num_words=num_words, formatted=False)
    for i, topic in enumerate(tqdm(topics, desc="Displaying Topics")):
        topic_words = [word for word, _ in topic[1]]  # topic[1]이 단어-가중치 리스트임
        print(f"Topic {i}: {topic_words}")

# 사용 예시
dictionary, corpus = prepare_data_for_lda(processed_docs)
lda_model = train_lda_model(dictionary, corpus, num_topics=5, passes=15)
print_lda_topics(lda_model)

Creating Corpus: 100%|██████████| 6/6 [00:00<00:00, 81.54it/s]


Training LDA Model...
토픽 추출 중...


Displaying Topics: 100%|██████████| 5/5 [00:00<00:00, 42974.43it/s]

Topic 0: ['고려아연', '서울', '상승', '시장', '투자', '파트너', '금리', '기업', '미국', '가격']
Topic 1: ['미국', '대통령', '이스라엘', '해리스', '트럼프', '후보', '부통령', '중국', '지난', '대선']
Topic 2: ['의원', '대통령', '국민', '여사', '민주당', '대표', '북한', '김건희', '공천', '지금']
Topic 3: ['의원', '대통령', '국민', '민주당', '여사', '대표', '서울', '대해', '대한', '관련']
Topic 4: ['경찰', '지난', '병원', '서울', '지역', '혐의', '위해', '기자', '사진', '대한']
